In [2]:
import numpy as np
import pandas as pd
import gensim
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences


from word2vec import classification_tools

Using TensorFlow backend.


In [3]:
RANDOM_STATE = 42

model_path = '/home/pierre/riken/word2vec/prot_vec_model.model'
data_path = '/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv'

As input we want list containing indexes.

# Data import and preprocessing

In [4]:
df = pd.read_csv(data_path, sep='\t')
df.loc[:, 'seq_len'] = df.sequences.apply(len)
df = df.loc[df.seq_len >= 50, :]

X, y = df['sequences'].values, df['is_allergenic'].values
X_tokens = classification_tools.ProteinTokenizer(token_size=3).transform(X)
lenghts = df.seq_len.values

In [30]:
model = gensim.models.word2vec.Word2Vec.load(model_path)

/home/pierre/.conda/envs/python3/lib/python3.5/site-packages/Bio/Seq.py:163: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [31]:
labels_idx = {'_UNKNOWN': 0}
embeddings = []

for sentence in X_tokens:
    sentence_embed = []
    for token in sentence:
        if token not in model.wv:
            token = '_UNKNOWN'
        if token not in labels_idx:
            labels_idx[token] = len(labels_idx)
        idx = labels_idx[token]
        sentence_embed.append(idx)
    embeddings.append(sentence_embed)

matrix_embeddings = np.zeros((len(labels_idx), 100))
labels_idx.pop('_UNKNOWN', None)
matrix_embeddings[1:] = model.wv[labels_idx.keys()]

embeddings = pad_sequences(embeddings, maxlen=300)

In [32]:
Xtrain, Xtest, ytrain, ytest = train_test_split(embeddings, y, test_size=0.3)

# Model Architecture

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import Adam
import keras.metrics

from keras.callbacks import TensorBoard, ModelCheckpoint

In [42]:
lstm_nn = Sequential()
lstm_nn.add(Embedding(len(matrix_embeddings), output_dim=100, weights=[matrix_embeddings], trainable=False))
lstm_nn.add(LSTM(10, return_sequences=True))
lstm_nn.add(Dropout(0.5))
lstm_nn.add(LSTM(10))
lstm_nn.add(Dense(1, activation='sigmoid'))

lstm_nn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [43]:
lstm_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 100)         817400    
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 10)          4440      
_________________________________________________________________
dropout_7 (Dropout)          (None, None, 10)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 822,691
Trainable params: 5,291
Non-trainable params: 817,400
_________________________________________________________________


In [44]:
tb = TensorBoard(log_dir='./logs4')
ckpt = ModelCheckpoint(filepath='./logs4')
lstm_nn.fit(Xtrain, ytrain, batch_size=128, epochs=49, validation_data=(Xtest, ytest), 
          callbacks=[tb]
         )

Train on 8729 samples, validate on 3741 samples
Epoch 1/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.5075 - acc: 0.8443 - val_loss: 0.4379 - val_acc: 0.8415
Epoch 2/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.4328 - acc: 0.8443 - val_loss: 0.4371 - val_acc: 0.8415
Epoch 3/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.4330 - acc: 0.8443 - val_loss: 0.4370 - val_acc: 0.8415
Epoch 4/49
8729/8729 [==============================] - 48s 5ms/step - loss: 0.4330 - acc: 0.8443 - val_loss: 0.4368 - val_acc: 0.8415
Epoch 5/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.4326 - acc: 0.8443 - val_loss: 0.4367 - val_acc: 0.8415
Epoch 6/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.4320 - acc: 0.8443 - val_loss: 0.4365 - val_acc: 0.8415
Epoch 7/49
8729/8729 [==============================] - 47s 5ms/step - loss: 0.4323 - acc: 0.8443 - val_loss: 0.4361 - val_acc: 0.8415
Epoch 8

In [45]:
!ls logs

ls: cannot access 'logs': No such file or directory


In [47]:
ypred = lstm_nn.predict_proba(Xtest)

In [51]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(ytest, ypred >= 0.5))

print('ROC AUC SCORE: ', roc_auc_score(ytest, ypred))

             precision    recall  f1-score   support

      False       0.89      0.98      0.94      3148
       True       0.81      0.37      0.51       593

avg / total       0.88      0.89      0.87      3741

ROC AUC SCORE:  0.7809005316151372
